<p style="font-family: helvetica,arial,sans-serif; font-size:2.0em;color:white; background-color: black">&emsp;<b>Pedestrian Counting System - Past Hour (counts per minute) </b></p>
    
<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:black; background-color: #DDDDDD; text-align:justify">&emsp;<b>Authored by: </b> Prathibha Kandawala and Jackson Tran</p>

<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:white; background-color: black; text-align:right"><b>&emsp;</p>

<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:black; background-color: #DDDDDD; text-align:justify">&emsp;<b>Level: </b>Intermediate&emsp;&emsp;<b>Pre-requisite Skills:</b>&emsp;&emsp;Python, Data analysis</p>

<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:white; background-color: black">&emsp;<b>Scenario</b>

In the heart of Melbourne, one of Australia's most vibrant and bustling cities, a sophisticated Pedestrian Counting System has been deployed. This system utilizes an extensive network of sensors strategically placed across the city to capture real-time data on pedestrian movement. One specific use case of this system is the analysis of past-hour pedestrian counts per minute, providing valuable insights into the dynamic patterns of foot traffic in Melbourne's urban landscape.

<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:white; background-color: black">&emsp;<b>Objectives </b>

The primary objective of this use case is to gain a deeper understanding of how pedestrian activity fluctuates on an hourly basis. By examining past-hour counts per minute, the City of Melbourne aims to:

**Temporal Analysis:** Study pedestrian behavior throughout the day to identify peak hours, trends, and daily variations.

**Event Impact:** Assess the impact of major events, weather conditions, or other external factors on pedestrian flow.

**Resource Allocation:** Make informed decisions about the allocation of resources for urban planning, traffic management, and emergency response.

<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:white; background-color: black">&emsp;<b>Strategic benefits for City of Melbourne
</b>

This use case offers several strategic benefits to the City of Melbourne and its stakeholders:

**Evidence-Based Decision Making:** By leveraging real-time data, the city can make data-driven decisions for urban planning and management, enhancing the overall quality of life for its residents and visitors.

**Improved Safety:** The analysis of pedestrian activity patterns supports efforts to enhance pedestrian safety and prioritize safety measures where they are most needed.

**Efficient Resource Allocation:** The ability to identify peak pedestrian hours and factors influencing traffic flow aids in optimizing resource allocation, whether for events, transportation, or emergency response.

**Economic Impact Assessment:** Understanding pedestrian behavior helps assess the economic impact of foot traffic on local businesses and facilitates strategic decisions related to retail development.

<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:white; background-color: black">&emsp;<b>Background</b>


 The Pedestrian Counting System in Melbourne is a testament to the city's commitment to creating a safe, vibrant, and dynamic urban environment. It consists of an extensive network of sensors strategically placed across the city, recording multi-directional pedestrian movements around the clock, without compromising individual privacy.

These sensors transmit data to a central server, which is then made accessible to the public through an online visualization tool. This tool enables users to analyze and compare pedestrian volume data for different locations and times, including the past-hour counts per minute, offering a wealth of information for various stakeholders.

In this use case, we delve into the significance of the "Pedestrian Counting System - Past Hour (counts per minute)" data and how it contributes to informed decision-making, safety enhancement, and economic prosperity in the City of Melbourne.

In [ ]:
import requests
import pandas as pd
import keys as keys

In [ ]:
API_KEY = keys.API_KEY

In [ ]:
BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'

NUMBER_OF_RECORDS = 100
#filter
FILTERS = f'records?limit={NUMBER_OF_RECORDS}&offset=0&timezone=UTC'

#Dataset name from URL
PEDESTRIAN_LOCATION_DATASET = 'pedestrian-counting-system-sensor-locations'
MICROCLIMATE_SENSOR_LOCATION_DATASET = 'microclimate-sensor-locations'

#Dataset name from excel
PEDESTRIAN_COUNTING_DATASET = pd.read_csv('PedestrianSafety/dependencies/pedestrian-counting-system-past-hour-counts-per-minute.csv')
MICROCLIMATE_SENSOR_DATASET = pd.read_csv('PedestrianSafety/dependencies/microclimate-sensor-readings.csv')
FOOTPATH_STEEPNESS_DATASET = pd.read_csv('PedestrianSafety/dependencies/footpath-steepness.csv')

PEDESTRIAN_LOCATION_URL = f'{BASE_URL}{PEDESTRIAN_LOCATION_DATASET}/{FILTERS}'
MICROCLIMATE_LOCATION_URL = f'{BASE_URL}{MICROCLIMATE_SENSOR_LOCATION_DATASET}/{FILTERS}'


In [ ]:
s = requests.Session()

In [ ]:
def get_data(base, data_url, size = 0):
    filters = f'records?limit={10}&offset={size}&timezone=UTC'
    url = f'{base}{data_url}/{filters}'
    result=s.get(url +f'&apikey={API_KEY}')
    result_json = result.json()
    max_results = result_json['total_count']
    records = result_json['results']
    records_df = pd.json_normalize(records)
    
    column_names = records_df.columns.values.tolist()
    
    next_url = None
    if size != max_results:
        
        size = records_df.shape[0]
        next_url = f'{base}{data_url}/{filters}'
    return [records_df, next_url, column_names]

In [ ]:
Pesestrian_df_new, Pedestrian_url_next, Pedestrian_column_names = get_data(BASE_URL,PEDESTRIAN_LOCATION_DATASET)
PEDESTRIAN_LOCATION_DF = pd.DataFrame(columns=Pedestrian_column_names)

In [ ]:
def add_to_dataframe(dataframe,new_data):
    df = pd.concat([dataframe,new_data])
    return df

In [ ]:
PEDESTRIAN_LOCATION_DF = add_to_dataframe(PEDESTRIAN_LOCATION_DF, Pesestrian_df_new)


In [ ]:
while Pedestrian_url_next != None:
    Pesestrian_df_new, Pedestrian_url_next, Pedestrian_column_names = get_data(BASE_URL, PEDESTRIAN_LOCATION_DATASET,PEDESTRIAN_LOCATION_DF.shape[0])
    PEDESTRIAN_LOCATION_DF = add_to_dataframe(PEDESTRIAN_LOCATION_DF, Pesestrian_df_new)


In [ ]:
PEDESTRIAN_LOCATION_DF.head()

In [ ]:
PEDESTRIAN_LOCATION_DF.shape

In [ ]:
Microclimate_df_new, Microclimate_url_next, Microclimate_column_names = get_data(BASE_URL,MICROCLIMATE_SENSOR_LOCATION_DATASET)
MICROCLIMATE_LOCATION_DF = pd.DataFrame(columns=Microclimate_column_names)

In [ ]:
MICROCLIMATE_LOCATION_DF = add_to_dataframe(MICROCLIMATE_LOCATION_DF, Microclimate_df_new)

In [ ]:
while Microclimate_url_next != None:
    Microclimate_df_new, Microclimate_url_next, Microclimate_column_names = get_data(BASE_URL, MICROCLIMATE_SENSOR_LOCATION_DATASET,MICROCLIMATE_LOCATION_DF.shape[0])
    MICROCLIMATE_LOCATION_DF = add_to_dataframe(MICROCLIMATE_LOCATION_DF, Microclimate_df_new)

In [ ]:
MICROCLIMATE_LOCATION_DF.head()

In [ ]:
MICROCLIMATE_LOCATION_DF.shape

In [ ]:
PEDESTRIAN_COUNTING_DATASET.head()

In [ ]:
PEDESTRIAN_COUNTING_DATASET.shape

In [ ]:
MICROCLIMATE_SENSOR_DATASET.head()

In [ ]:
MICROCLIMATE_SENSOR_DATASET.shape

In [ ]:
FOOTPATH_STEEPNESS_DATASET.head()

In [ ]:
FOOTPATH_STEEPNESS_DATASET.shape